In [1]:
%matplotlib inline
from config_calc import *
from glob import glob
import matplotlib.pyplot as plt
plt.rcParams['mathtext.default']='regular'
import matplotlib.colors as colors

import cmocean
import colorbrewer
import plot_tools

from config_calc import *
from proc_cesm_le import open_ens
                                     
varlist = ['O2','AOU'] #,'TEMP']
op = 'ann_dft'
tr85 = {}; tr85a = {}
tr45 = {}; tr45a = {}
ctrl = {}

#-- read datasets
tr85 = open_ens('tr85',op,varlist)
tr45 = open_ens('tr85',op,varlist)

#-- compute derived
v = 'O2sat'
tr85[v] = tr85['AOU'] + tr85['O2']
tr45[v] = tr45['AOU'] + tr45['O2']
varlist.append(v)  

tr85a = tr85.mean(dim='ens')
tr45a = tr45.mean(dim='ens')

tr85a

/glade/p/work/mclong/miniconda2/lib/python2.7/site-packages/dask/array/numpy_compat.py:45: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


<xarray.Dataset>
Dimensions:             (d2: 2, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 182, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                (time) float64 7.01e+05 7.01e+05 7.014e+05 7.017e+05 ...
  * z_t                 (z_t) float64 500.0 1.5e+03 2.5e+03 3.5e+03 4.5e+03 ...
    ULAT                (nlat, nlon) float64 -78.95 -78.95 -78.95 -78.95 ...
    ULONG               (nlat, nlon) float64 321.1 322.3 323.4 324.5 325.6 ...
  * lat_aux_grid        (lat_aux_grid) float32 -79.4882 -78.9529 -78.4187 ...
  * moc_z               (moc_z) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
  * z_t_150m            (z_t_150m) float32 500.0 1500.0 2500.0 3500.0 4500.0 ...
  * z_w                 (z_w) float32 0.0 1000.0 2000.0 3000.0 4000.0 5000.0 ...
  * z_w_bot             (z_w_bot) float32 1000.0 2000.0 3000.0 4000.0 5000.0 ...
  * z_w_top             (z_w_top) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
Dimensions without coordi

In [2]:
tr85a_tc = tr85a.sel(z_t=slice(200.*100.,600.*100.)).mean(dim='z_t')
tr85a_tc

<xarray.Dataset>
Dimensions:             (d2: 2, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 182, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                (time) float64 7.01e+05 7.01e+05 7.014e+05 7.017e+05 ...
    ULAT                (nlat, nlon) float64 -78.95 -78.95 -78.95 -78.95 ...
    ULONG               (nlat, nlon) float64 321.1 322.3 323.4 324.5 325.6 ...
  * lat_aux_grid        (lat_aux_grid) float32 -79.4882 -78.9529 -78.4187 ...
  * moc_z               (moc_z) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
  * z_t_150m            (z_t_150m) float32 500.0 1500.0 2500.0 3500.0 4500.0 ...
  * z_w                 (z_w) float32 0.0 1000.0 2000.0 3000.0 4000.0 5000.0 ...
  * z_w_bot             (z_w_bot) float32 1000.0 2000.0 3000.0 4000.0 5000.0 ...
  * z_w_top             (z_w_top) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    ANGLE               (nlat, nlon) float64 0.0 0.